In [95]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st 
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [158]:
tk_data = pd.read_csv('~/Desktop/CraftIQ/Projects/GitHub/AnnieTracy_Streamlit/Data/z_RegressionSocialData.csv')
x = tk_data.drop('Profile Views', axis =1)
y = tk_data['Profile Views']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 123)

In [159]:
x_train_p = np.array(x_train)

In [160]:
y_train_p = np.array(y_train).ravel()

In [161]:
y_test.count()

10

In [162]:
x_test.count()

Video Views       10
Likes             10
Comments          10
Shares            10
Unique Viewers    10
dtype: int64

In [163]:
x_train.count()

Video Views       37
Likes             37
Comments          37
Shares            37
Unique Viewers    37
dtype: int64

In [164]:
lasso_cv = LassoCV() #ls is the container for parameter m and c in y=mX+c

lasso_cv.fit(x_train_p , y_train_p) #training part to get m and c

LassoCV()

In [168]:
c = lasso_cv.intercept_.round(0)

m = lasso_cv.coef_.round(0)

print(c)
print(m)

3189.0
[11549. -1876.    -0.   463. -7074.]


In [149]:
y_pred_test = m*x_test + c

In [150]:
y_pred_test

,Video Views,Likes,Comments,Shares,Unique Viewers
8,17008.790775,2588.165250,3235.911046,3387.350183,39.043159
11,32141.016435,-1427.247711,3235.911046,4168.074669,-15062.671900
44,-4926.338096,4303.502095,3235.911046,2904.143162,8005.071172
18,5077.638532,2976.974935,3235.911046,3138.153419,2037.225406
43,-5819.754081,4458.009473,3235.911046,2882.743994,8740.594011


In [153]:
random_indices = np.random.choice(len(x_test), 5, replace=False)

In [154]:
x_samples = x_train.iloc[random_indices]

In [155]:
x_samples

,Video Views,Likes,Comments,Shares,Unique Viewers
42,-0.793621,-0.729958,-0.635859,-0.746713,-0.796796
5,-0.190001,-0.266240,0.002743,0.281357,-0.040992
31,1.224969,0.732300,-0.213122,-0.191080,1.882777
10,3.367347,3.678628,2.811992,2.723764,3.516060
1,0.706620,1.016090,2.620111,2.609369,0.837073


In [156]:
y_pred_samples = y_pred_test.iloc[random_indices]

In [157]:
y_pred_samples 

,Video Views,Likes,Comments,Shares,Unique Viewers
11,32141.016435,-1427.247711,3235.911046,4168.074669,-15062.671900
18,5077.638532,2976.974935,3235.911046,3138.153419,2037.225406
8,17008.790775,2588.165250,3235.911046,3387.350183,39.043159
44,-4926.338096,4303.502095,3235.911046,2904.143162,8005.071172
43,-5819.754081,4458.009473,3235.911046,2882.743994,8740.594011
